In [1]:
import numpy as np
from scipy import sparse
import pandas as pd
import xgboost as xgb
import re
import string
import time
from bayes_opt import BayesianOptimization

from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import feature_selection
from itertools import product
%matplotlib inline 

In [2]:
train_data = pd.read_json('../input/train.json')
test_data = pd.read_json('../input/test.json')

In [3]:
train_size = train_data.shape[0]

### Create target variables

In [4]:
train_data['target'] = train_data['interest_level'].apply(lambda x: 0 if x=='low' else 1 if x=='medium' else 2)
train_data['low'] = train_data['interest_level'].apply(lambda x: 1 if x=='low' else 0)
train_data['medium'] = train_data['interest_level'].apply(lambda x: 1 if x=='medium' else 0)
train_data['high'] = train_data['interest_level'].apply(lambda x: 1 if x=='high' else 0)

## Merge training and testing data

In [5]:
full_data=pd.concat([train_data
                       ,test_data])

### Group Variables

In [6]:
num_vars = ['bathrooms','bedrooms','latitude','longitude','price']
cat_vars = ['building_id','manager_id','display_address','street_address']
text_vars = ['description','features']
date_var = 'created'
image_var = 'photos'
id_var = 'listing_id'

In [7]:
full_data.head(5)

,bathrooms,bedrooms,building_id,created,description,display_address,features,high,interest_level,latitude,listing_id,longitude,low,manager_id,medium,photos,price,street_address,target
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],0.0,medium,40.7145,7211212,-73.9425,0.0,5ba989232d0489da1b5f2c45f6688adc,1.0,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,1.0
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",0.0,low,40.7947,7150865,-73.9667,1.0,7533621a882f71e25173b27e3139d83d,0.0,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,0.0
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",1.0,high,40.7388,6887163,-74.0018,0.0,d9039c43983f6e564b1482b273bd7b01,0.0,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,2.0
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",0.0,low,40.7539,6888711,-73.9677,1.0,1067e078446a7897d2da493d2f741316,0.0,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,0.0
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],0.0,low,40.8241,6934781,-73.9493,1.0,98e13ad4b495b9613cef886d79a6291f,0.0,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,0.0


### Date variable

In [8]:
full_data['created_datetime'] = pd.to_datetime(full_data['created'], format="%Y-%m-%d %H:%M:%S")

In [9]:
# full_data['created_year']=full_data['created_datetime'].apply(lambda x:x.year) ## low variant
full_data['created_month']=full_data['created_datetime'].apply(lambda x:x.month)
full_data['created_day']=full_data['created_datetime'].apply(lambda x:x.day)
full_data['created_dayofweek']=full_data['created_datetime'].apply(lambda x:x.dayofweek)
full_data['created_dayofyear']=full_data['created_datetime'].apply(lambda x:x.dayofyear)
full_data['created_weekofyear']=full_data['created_datetime'].apply(lambda x:x.weekofyear)
full_data['created_hour']=full_data['created_datetime'].apply(lambda x:x.hour)
full_data['created_epoch']=full_data['created_datetime'].apply(lambda x:x.value//10**9)

date_num_vars = ['created_month','created_dayofweek','created_dayofyear'
                 ,'created_weekofyear','created_hour','created_epoch']

## Additional Numeric Variables

In [10]:
# full_data['price']=full_data['price'].apply(np.log)

In [11]:
full_data['rooms'] = full_data['bedrooms'] + full_data['bathrooms'] 
full_data['num_of_photos'] = full_data['photos'].apply(lambda x:len(x))
full_data['num_of_features'] = full_data['features'].apply(lambda x:len(x))
full_data['len_of_desc'] = full_data['description'].apply(lambda x:len(x))
full_data['words_of_desc'] = full_data['description'].apply(lambda x:len(re.sub('['+string.punctuation+']', '', x).split()))


full_data['nums_of_desc'] = full_data['description']\
        .apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: len([s for s in x if s.isdigit()]))
        
full_data['has_phone'] = full_data['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)
full_data['has_email'] = full_data['description'].apply(lambda x: 1 if '@renthop.com' in x else 0)

additional_num_vars = ['rooms','num_of_photos','num_of_features','len_of_desc',
                    'words_of_desc','has_phone','has_email']

### Numeric interactions

In [12]:
full_data['avg_word_len'] = full_data[['len_of_desc','words_of_desc']]\
                                    .apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
    
full_data['price_per_room'] = full_data[['price','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_bedroom'] = full_data[['price','bedrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_bathroom'] = full_data[['price','bathrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_photo'] = full_data[['price','num_of_photos']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)


full_data['photos_per_room'] = full_data[['num_of_photos','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)

interactive_num_vars = ['avg_word_len','price_per_room','price_per_bedroom','price_per_bathroom','price_per_photo',
                        'photos_per_room']

### Categorical Features

#### 1. Label Encoding

In [13]:
LBL = preprocessing.LabelEncoder()

LE_vars=[]
LE_map=dict()
for cat_var in cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var])
    LE_vars.append(LE_var)
    LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (LE_vars))

Label Encoding building_id
Label Encoding manager_id
Label Encoding display_address
Label Encoding street_address
Label-encoded feaures: ['building_id_le', 'manager_id_le', 'display_address_le', 'street_address_le']


#### 2. One Hot Encoding

In [14]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
OHE.fit(full_data[LE_vars])
OHE_sparse=OHE.transform(full_data[LE_vars])
                                   
print ('One-hot-encoding finished in %f seconds' % (time.time()-start))


OHE_vars = [var[:-3] + '_' + level.encode('utf-8').replace(' ','_')\
                for var in cat_vars for level in LE_map[var] ]

print ("OHE_sparse size :" ,OHE_sparse.shape)
print ("One-hot encoded catgorical feature samples : %s" % (OHE_vars[:100]))

One-hot-encoding finished in 0.310443 seconds
OHE_sparse size : (124011, 57868)
One-hot encoded catgorical feature samples : ['building_0', 'building_00005cb939f9986300d987652c933e15', 'building_00024d77a43f0606f926e2312513845c', 'building_000ae4b7db298401cdae2b0ba1ea8146', 'building_0012f1955391bca600ec301035b97b65', 'building_0021440c04241281a436ec21accc40b1', 'building_002d1eba40aa0a6610e04ff20543585f', 'building_003d8740e21484dcc2280639b25539a4', 'building_00480e54b53fe77d17964be3f8307a99', 'building_00553d95d22484bcc36831c9248d1dbc', 'building_0055c8662ba19e95f78df97592d2b83e', 'building_0056dbdf2881b76f2a0171eb753ec9e0', 'building_0059ae562b9e338a59eaf962cb3eedd2', 'building_005e0f8d7fb7b92be351cbf1dd985149', 'building_0067f166111490e7af7f1a878a67bb5e', 'building_0070bc94a3f80aa717bb15708e98ba54', 'building_0071cda335745940cdae1dc31abfc701', 'building_0078281cd69f4bfec17e42e5cf5eecd9', 'building_0078c2ab46afba9969637ac83621901e', 'building_007ae1cd90420f18bad7b6892a9a1411', 'buil

#### 3. Leave-one-out Encoding

Based on the paper "A Preprocessing Scheme for High-Cardinality Categorical Attributes in Classification and Prediction Problems"

http://helios.mm.di.uoa.gr/~rouvas/ssi/sigkdd/sigkdd.vol3.1/barreca.ps

** A couple of Kaggle scripts: **

R version: by Braden Murray: https://www.kaggle.com/brandenkmurray/two-sigma-connect-rental-listing-inquiries/it-is-lit/comments

Python Version 1, by Stanislav Ushakov
https://www.kaggle.com/stanislavushakov/two-sigma-connect-rental-listing-inquiries/python-version-of-it-is-lit-by-branden/comments

Python Version 2, by Rakhlin
https://www.kaggle.com/rakhlin/two-sigma-connect-rental-listing-inquiries/another-python-version-of-it-is-lit-by-branden/code


In [15]:
##Create a function to encode high-cardinality cateogrical features

def designate_single_observations(df1, df2, column):
    ps = df1[column].append(df2[column])
    grouped = ps.groupby(ps).size().to_frame().rename(columns={0: "size"})
    df1.loc[df1.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    df2.loc[df2.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    return df1, df2


def hcc_encode(train_df, test_df, variable, target, prior_prob, k, f=1, g=1, r_k=None, update_df=None):
    """
    See "A Preprocessing Scheme for High-Cardinality Categorical Attributes in
    Classification and Prediction Problems" by Daniele Micci-Barreca
    """
    hcc_name = "_".join(["hcc", variable, target])

    grouped = train_df.groupby(variable)[target].agg({"size": "size", "mean": "mean"})
    grouped["lambda"] = 1 / (g + np.exp((k - grouped["size"]) / f))
    grouped[hcc_name] = grouped["lambda"] * grouped["mean"] + (1 - grouped["lambda"]) * prior_prob

    df = test_df[[variable]].join(grouped, on=variable, how="left")[hcc_name].fillna(prior_prob)
    if r_k: df *= np.random.uniform(1 - r_k, 1 + r_k, len(test_df))     # Add uniform noise. Not mentioned in original paper

    if update_df is None: update_df = test_df
    if hcc_name not in update_df.columns: update_df[hcc_name] = np.nan
    update_df.update(df)
    return

In [16]:
for col in ('building_id', 'manager_id', 'display_address'):
    train_data, test_data = designate_single_observations(train_data, test_data, col)
    
prior_low, prior_medium, prior_high = train_data[["low", "medium", "high"]].mean() 

skf = model_selection.StratifiedKFold(5)
attributes = product(("building_id", "manager_id"), zip(("medium", "high"), (prior_medium, prior_high)))
for variable, (target, prior) in attributes:
    hcc_encode(train_data, test_data, variable, target, prior, k=5, r_k=None)
    for train, test in skf.split(np.zeros(len(train_data)), train_data['interest_level']):
        hcc_encode(train_data.iloc[train], train_data.iloc[test], variable, target, prior, k=5, r_k=0.01,
                   update_df=train_data)
        
hcc_data = pd.concat([train_data[['building_id', 'manager_id', 'display_address',
            'hcc_building_id_medium','hcc_building_id_high',
            'hcc_manager_id_medium','hcc_manager_id_high']],
           test_data[['building_id', 'manager_id', 'display_address',
            'hcc_building_id_medium','hcc_building_id_high',
            'hcc_manager_id_medium','hcc_manager_id_high']]
           ]
          )
full_data['building_id'] = hcc_data['building_id']
full_data['manager_id'] = hcc_data['manager_id']
full_data['display_address'] = hcc_data['display_address']
full_data['hcc_building_id_medium'] = hcc_data['hcc_building_id_medium']
full_data['hcc_building_id_high'] = hcc_data['hcc_building_id_high']
full_data['hcc_manager_id_medium'] = hcc_data['hcc_manager_id_medium']
full_data['hcc_manager_id_high'] = hcc_data['hcc_manager_id_high']
hcc_vars = ['hcc_building_id_medium','hcc_building_id_high','hcc_manager_id_medium','hcc_manager_id_high']    

### Text Features

#### 1. Features

In [17]:
full_data["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
cntvec = CountVectorizer(stop_words='english', max_features=200)
feature_sparse =cntvec.fit_transform(full_data["features"]\
                                     .apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x])))

feature_vars = ['feature_' + v for v in cntvec.vocabulary_]

#### 2. Description

##### tf-idf not working, instead, using CountVectorizer, not working either

In [18]:
# tfidf = TfidfVectorizer(stop_words='english', max_features=10)
# desc_sparse = tfidf.fit_transform(full_data["description"])
# desc_vars = ['desc_' + v for v in tfidf.get_feature_names()]

In [19]:
cntvec = CountVectorizer(stop_words='english', max_features=100)
desc_sparse = cntvec.fit_transform(full_data["description"])
desc_vars = ['desc_' + v for v in cntvec.vocabulary_]

##### word2vec - to be added

#### Street Address

In [20]:
cntvec = CountVectorizer(stop_words='english', max_features=10)
st_addr_sparse = cntvec.fit_transform(full_data["street_address"])
st_addr_vars = ['desc_' + v for v in cntvec.vocabulary_]

#### image vars - to be added

### Numberic vs Categorical Interactions

In [21]:
price_by_manager = full_data.groupby('manager_id')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_manager.columns = ['manager_id','min_price_by_manager',
                            'max_price_by_manager','median_price_by_manager','mean_price_by_manager']
full_data = pd.merge(full_data,price_by_manager, how='left',on='manager_id')

price_by_building = full_data.groupby('building_id')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_building.columns = ['building_id','min_price_by_building',
                            'max_price_by_building','median_price_by_building','mean_price_by_building']
full_data = pd.merge(full_data,price_by_building, how='left',on='building_id')

price_by_disp_addr = full_data.groupby('display_address')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_disp_addr.columns = ['display_address','min_price_by_disp_addr',
                            'max_price_by_disp_addr','median_price_by_disp_addr','mean_price_by_disp_addr']
full_data = pd.merge(full_data,price_by_disp_addr, how='left',on='display_address')

num_cat_vars = ['median_price_by_manager','mean_price_by_manager',
                'median_price_by_building','mean_price_by_building',
                'median_price_by_disp_addr','mean_price_by_disp_addr'
               ]

full_data['price_percentile_by_manager']=\
            full_data[['price','min_price_by_manager','max_price_by_manager']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
full_data['price_percentile_by_building']=\
            full_data[['price','min_price_by_building','max_price_by_building']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
full_data['price_percentile_by_disp_addr']=\
            full_data[['price','min_price_by_disp_addr','max_price_by_disp_addr']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)

num_cat_vars.append('price_percentile_by_manager')
num_cat_vars.append('price_percentile_by_building')
num_cat_vars.append('price_percentile_by_disp_addr')

### Listing ID matters

In [22]:
min_listing_id = full_data['listing_id'].min()
max_listing_id = full_data['listing_id'].max()
full_data['listing_id_pos']=full_data['listing_id'].apply(lambda x:np.float64((x-min_listing_id+1))/(max_listing_id-min_listing_id+1))
num_vars.append('listing_id')
num_vars.append('listing_id_pos')

### Create training and testing data

In [23]:
%%time
##Baseline with features from "features" and street address

full_vars = num_vars + date_num_vars + additional_num_vars + interactive_num_vars + LE_vars + hcc_vars + num_cat_vars

train_x = sparse.hstack([full_data[full_vars], feature_sparse,st_addr_sparse]).tocsr()[:train_size]
train_y = full_data['target'][:train_size].values

test_x = sparse.hstack([full_data[full_vars], feature_sparse,st_addr_sparse]).tocsr()[train_size:]
test_y = full_data['target'][train_size:].values


full_vars = full_vars + feature_vars + st_addr_vars
print ("training data size: ", train_x.shape,"testing data size: ", test_x.shape)

training data size:  (49352, 253) testing data size:  (74659, 253)
CPU times: user 2.23 s, sys: 481 ms, total: 2.71 s
Wall time: 2.71 s
